Install TreeInterpreter <br>
[Github](https://github.com/andosa/treeinterpreter) <br>
[Blog Explainer](http://blog.datadive.net/interpreting-random-forests/)


In [1]:
!pip install treeinterpreter

Setting up


In [2]:
from treeinterpreter import treeinterpreter as ti
from sklearn.ensemble import RandomForestRegressor
import numpy as np

 #Load Dataset
from sklearn.datasets import load_boston
boston = load_boston()
#Set model type as Random Forest Regressor
rf = RandomForestRegressor()

#Calculating bias and feature prediction for two rows of data

In [ ]:
# Lets use the first 300 rows of data
X = boston.data[:300] # Input Features
y = boston.target[:300] # Target Vector
rf.fit(X, y) # Fit the model on X and y

In [7]:
instances = boston.data[[300, 309]]
#print(instances) # Two rows of houses
print ("Instance 0 prediction:", rf.predict([instances[0]]))
print ("Instance 1 prediction:", rf.predict([instances[1]]))

Instance 0 prediction: [29.851]
Instance 1 prediction: [22.466]


In [8]:
# Use TreeInterpreter to caluclate bias and contributions
prediction, bias, contributions = ti.predict(rf, instances)
# Finding bias and contributions for each prediction 
for i in range(len(instances)):
    print ("Instance", i)
    print ("Bias (trainset mean)", bias[i])
    print ("Feature contributions:")
    for contri, feature in sorted(zip(contributions[i], 
                                 boston.feature_names), 
                             key=lambda x: -abs(x[0])):
        print (feature, round(contri, 2))
    print ("-"*20 )

Instance 0
Bias (trainset mean) 25.55410666666667
Feature contributions:
RM 3.05
LSTAT 1.46
PTRATIO 0.53
TAX -0.52
DIS -0.3
INDUS 0.21
ZN 0.2
CRIM -0.17
NOX -0.14
B -0.11
AGE 0.05
RAD 0.04
CHAS -0.0
--------------------
Instance 1
Bias (trainset mean) 25.55410666666667
Feature contributions:
RM -5.4
LSTAT 2.25
CRIM 0.42
AGE -0.2
TAX -0.2
B -0.17
PTRATIO 0.09
RAD 0.05
DIS 0.04
NOX 0.03
INDUS 0.02
CHAS -0.01
ZN 0.0
--------------------


Verifiying the methodology by - <br>
Bias + Contribution = Prediction <br>
Which proves correct. ( As seen by comapring with previously calculated values)

In [12]:
print ("Prediction: ",prediction)
print ("Addition: ",bias + np.sum(contributions, axis=1))

Prediction:  [[29.851]
 [22.466]]
Addition:  [29.851 22.466]


# Calculating feature contributions of the whole model


Splitting dataset into two (~100 rows)



In [13]:
ds1 = boston.data[300:400]
ds2 = boston.data[400:]
 
print (np.mean(rf.predict(ds1)))
print (np.mean(rf.predict(ds2)))

22.15354
18.142566037735847


Predicting biases and contributions for both

In [14]:
prediction1, bias1, contributions1 = ti.predict(rf, ds1)
prediction2, bias2, contributions2 = ti.predict(rf, ds2)

Calculcating average contributions for each feature

In [15]:
totalc1 = np.mean(contributions1, axis=0) 
totalc2 = np.mean(contributions2, axis=0) 

Verifying

In [16]:
print (np.sum(totalc1 - totalc2))
print (np.mean(prediction1) - np.mean(prediction2))

4.010973962264151
4.010973962264153


The sum of the feature contribution differences should be equal to the difference in average prediction

In [17]:
for c, feature in sorted(zip(totalc1 - totalc2, 
                             boston.feature_names), reverse=True):
    print (feature, round(c, 2))

LSTAT 2.66
CRIM 0.63
RM 0.47
NOX 0.11
PTRATIO 0.09
AGE 0.08
B 0.07
TAX 0.02
ZN 0.01
INDUS 0.0
CHAS -0.01
RAD -0.03
DIS -0.1
